In [2]:
from sklearn.datasets import load_iris, load_diabetes
import pandas as pd
import numpy as np

In [62]:
# Load the data iris
iris = load_iris()
iris_X = iris.data
iris_Y = iris.target


# Load the data diabetes
diabetes = load_diabetes()
diabetes_X = diabetes.data
diabetes_Y = diabetes.target

In [5]:
### Load the data Titanic
titanic = pd.read_csv('./data/titanic/train.csv')
titanic_test = pd.read_csv('./data/titanic/test.csv')

In [6]:
titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [7]:
titanic_Y_df=titanic.loc[:, titanic.columns.isin(['Survived'])]
titanic_X=titanic.loc[:, ~titanic.columns.isin(['Survived',"Name","Ticket","Cabin","PassengerId","Embarked"])]
titanic_X_test=titanic_test.loc[:, ~titanic_test.columns.isin(['Survived',"Name","Ticket","Cabin","PassengerId","Embarked"])]

In [8]:
def one_hot_encoding(df, col):
    new_col = pd.get_dummies(df[col], prefix=col)
    df = pd.concat([df,new_col], axis=1).drop([col], axis=1)
    return df

def fill_na_mean(df, col):
    df[col].fillna(df[col].mean(), inplace=True)

titanic_X = one_hot_encoding(titanic_X,"Sex")
titanic_X_df = one_hot_encoding(titanic_X,"Pclass")
for col in titanic_X_df:
    fill_na_mean(titanic_X_df, col)

titanic_X_test = one_hot_encoding(titanic_X_test,"Sex")
titanic_X_test_df = one_hot_encoding(titanic_X_test,"Pclass")
for col in titanic_X_test_df:
    fill_na_mean(titanic_X_test_df, col)

In [9]:
#[titanic_X_df[col].isnull().values.any() for col in titanic_X_df]
[titanic_X_test_df[col].isnull().values.any() for col in titanic_X_test_df]

[False, False, False, False, False, False, False, False, False]

In [11]:
#Y=np.array([i for i in Y])
titanic_X=titanic_X_df.to_numpy()
titanic_X_test = titanic_X_test_df.to_numpy()
titanic_Y=titanic_Y_df.to_numpy().reshape(len(titanic_Y_df))

In [12]:
titanic_X[:5]

array([[22.    ,  1.    ,  0.    ,  7.25  ,  0.    ,  1.    ,  0.    ,
         0.    ,  1.    ],
       [38.    ,  1.    ,  0.    , 71.2833,  1.    ,  0.    ,  1.    ,
         0.    ,  0.    ],
       [26.    ,  0.    ,  0.    ,  7.925 ,  1.    ,  0.    ,  0.    ,
         0.    ,  1.    ],
       [35.    ,  1.    ,  0.    , 53.1   ,  1.    ,  0.    ,  1.    ,
         0.    ,  0.    ],
       [35.    ,  0.    ,  0.    ,  8.05  ,  0.    ,  1.    ,  0.    ,
         0.    ,  1.    ]])

In [13]:
titanic_X_df.columns, titanic_X_test_df.columns

(Index(['Age', 'SibSp', 'Parch', 'Fare', 'Sex_female', 'Sex_male', 'Pclass_1',
        'Pclass_2', 'Pclass_3'],
       dtype='object'),
 Index(['Age', 'SibSp', 'Parch', 'Fare', 'Sex_female', 'Sex_male', 'Pclass_1',
        'Pclass_2', 'Pclass_3'],
       dtype='object'))

In [41]:
## Utilities
def cal_gini(class_values, splits):
    '''calculate the gini index based on classes and data splits'''
    gini = 0.0
    total_size = 0
    
    for split in splits:
        size = len(split)
        if size == 0: continue
        for class_value in class_values:
            proportion = [row[-1] for row in split].count(class_value)/ float(size)
            gini += (proportion*(1.0-proportion))*size
        total_size += size

    return gini/total_size


def cal_variance(class_values, splits):
    total_variance = 0.0
    total_size = 0.0
    
    for split in splits:
        size = len(split)
        split_mean = np.mean([row[-1] for row in split])
        variance = sum([(row[-1]-split_mean)**2 for row in split])/ float(size-1)
        total_variance +=  variance*size
        total_size += size
    
    return total_variance/float(total_size)

def test_split(col_index, threshold, datasets):
    '''list all the potential splits for each col'''
    left, right = [], []
    
    for row in datasets:
        if row[col_index] < threshold: left.append(row)
        else: right.append(row)
            
    return left, right

def get_split(datasets, criterion=cal_gini):
    '''find the best split'''
    class_values = list(set(list([row[-1] for row in datasets])))
    best_col, best_threshold, best_score, best_group = float("inf"), float("inf"), float("inf"), None
    
    for col in range(len(datasets[0])-1):
        for row in range(len(datasets)):
            group = test_split(col, datasets[row][col], datasets)
            # use gini index to measure impurity here
            node_index = criterion(class_values, group)
            if node_index < best_score:
                best_col, best_threshold, best_score, best_group = \
                col, datasets[row][col], node_index, group
                
    return {'col_index': best_col, 'threshold': best_threshold, 'groups': best_group}

In [42]:
#### Testing ################
# Create child splits for a node or make terminal
def split(node, max_depth, min_size, depth):
    left, right = node['groups']
    del (node['groups'])
    # check for a no split
    if not left or not right:
        node['left'] = node['right'] = to_terminal(left + right)
        return
    # check for max depth
    if depth >= max_depth:
        node['left'], node['right'] = to_terminal(left), to_terminal(right)
        return
    # process left child
    if len(left) <= min_size:
        node['left'] = to_terminal(left)
    else:
        node['left'] = get_split(left, criterion=cal_variance)
        split(node['left'], max_depth, min_size, depth + 1)
    # process right child
    if len(right) <= min_size:
        node['right'] = to_terminal(right)
    else:
        node['right'] = get_split(right, criterion=cal_variance)
        split(node['right'], max_depth, min_size, depth + 1)

# Create a terminal node value
def to_terminal(group):
    outcomes = [row[-1] for row in group]
    return max(set(outcomes), key=outcomes.count)

# Build a decision tree
def build_tree(X, y, max_depth, min_size):
    datasets = np.column_stack([X, y.T])
    root = get_split(datasets)
    split(root, max_depth, min_size, 1)
    return root


# Print a decision tree
def print_tree(node, depth=0):
    if isinstance(node, dict):
        print('%s[X%d < %.3f]' % ((depth * ' ', (node['col_index'] + 1), node['threshold'])))
        print_tree(node['left'], depth + 1)
        print_tree(node['right'], depth + 1)
    else:
        print('%s[%s]' % ((depth * ' ', node)))

d1 = [[2.771244718, 1.784783929, 0.3],
   [1.728571309, 1.169761413, 0.6],
   [3.678319846, 2.81281357, 0.7],
   [3.961043357, 2.61995032, 0.7],
   [2.999208922, 2.209014212, 0.9],
   [7.497545867, 3.162953546, 1],
   [9.00220326, 3.339047188, 1.3],
   [7.444542326, 0.476683375, 1.1],
   [10.12493903, 3.234550982, 1],
   [6.642287351, 3.319983761, 1]]
d1 = np.array(d1)
d1_X = d1[:,:2]
d1_Y = d1[:,-1]
tree = build_tree(d1_X, d1_Y, 5, 1)
print_tree(tree)

[X2 < 3.163]
 [X1 < 2.999]
  [X1 < 1.729]
   [0.3]
   [0.3]
  [X2 < 2.620]
   [X1 < 2.999]
    [0.9]
    [0.9]
   [X1 < 3.678]
    [0.7]
    [0.7]
 [X1 < 9.002]
  [X1 < 6.642]
   [1.0]
   [1.0]
  [X1 < 9.002]
   [1.3]
   [1.3]


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [83]:
class MyDecisionTree:
    
    def __init__(self, min_samples_split=2, min_impurity=1e-7,
                 max_depth=float("inf"), loss=None, impurity_calculation=None):
        self.tree = None  # Root node in dec. tree
        # Minimum n of samples to justify split
        self.min_samples_split = min_samples_split
        # The minimum impurity to justify split
        self.min_impurity = min_impurity
        # The maximum depth to grow the tree to
        self.max_depth = max_depth
        # Function to calculate impurity (classif.=>info gain, regr=>variance reduct.)
        self._impurity_calculation = impurity_calculation
        # Function to determine prediction of y at leaf
        self._leaf_value_calculation = None
        # If Gradient Boost
        self.loss = loss
        
    def fit(self, X, y):
        self.tree = self._build_tree(X, y)
    
    def _build_tree(self, X, y):
        datasets = np.column_stack([X, y])
        root = get_split(datasets, criterion=self._impurity_calculation)
        split(root, self.max_depth, self.min_samples_split, 1)
        return root
        
    def predict_point(self, row, node=None):
        if not node: node = self.tree
        
        if row[node["col_index"]] < node["threshold"]:
            if isinstance(node["left"], dict):
                return self.predict_point(row, node["left"])
            else:
                return node["left"]
        else:
            if isinstance(node["right"], dict):
                return self.predict_point(row, node["right"])
            else:
                return node["right"]
    
    def predict(self, X_test, node=None):
        if not node: node = self.tree
        prediction = []
        
        for row in X_test:
            prediction.append(self.predict_point(row,self.tree))
        
        return np.array(prediction)

In [84]:
class MyDecisionTreeClassifier(MyDecisionTree):
    
    def __init__(self, min_samples_split=2, min_impurity=1e-7,
                 max_depth=float("inf"), loss=None, impurity_calculation=cal_gini):

        super().__init__(min_samples_split, min_impurity,
                 max_depth, impurity_calculation=impurity_calculation)

In [85]:
class MyDecisionTreeRegressor(MyDecisionTree):
    
    def __init__(self, min_samples_split=2, min_impurity=1e-7,
                 max_depth=float("inf"), loss=None, impurity_calculation=cal_variance):

        super().__init__(min_samples_split, min_impurity,
                 max_depth, impurity_calculation=impurity_calculation)

In [87]:
###################### Test Case #######################
d1 = [[2.771244718, 1.784783929, 0],
   [1.728571309, 1.169761413, 0],
   [3.678319846, 2.81281357, 0],
   [3.961043357, 2.61995032, 0],
   [2.999208922, 2.209014212, 0],
   [7.497545867, 3.162953546, 1],
   [9.00220326, 3.339047188, 1],
   [7.444542326, 0.476683375, 1],
   [10.12493903, 3.234550982, 1],
   [6.642287351, 3.319983761, 1]]
d1 = np.array(d1)
d1_X = d1[:,:2]
d1_Y = d1[:,-1]
d1_X_test = d1[:-2,:2]

d_clf = MyDecisionTreeClassifier(max_depth=5, min_samples_split=1)
d_clf.fit(d1_X,d1_Y)
prediction = d_clf.predict(d1_X_test)
print(prediction)

[0. 0. 0. 0. 0. 1. 1. 1.]


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in double_scalars


In [88]:
d_clf = MyDecisionTreeClassifier(max_depth=5, min_samples_split=2)
d_clf.fit(titanic_X,titanic_Y)
d_prediction = d_clf.predict(titanic_X_test)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in double_scalars


[0. 0. 0. 0. 1. 0. 1. 0. 1. 0. 0. 0. 1. 0. 1. 1. 0. 0. 1. 0. 0. 0. 1. 0.
 1. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 1. 0. 1. 0. 0. 0. 1. 1. 0. 0. 0.
 1. 1. 0. 0. 1. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 1. 1. 0. 0. 1. 1. 0.
 1. 0. 1. 0. 0. 1. 0. 1. 1. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 0. 1. 0. 0. 0.
 1. 0. 1. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 1. 0. 1.
 1. 0. 1. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0.
 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 1. 1. 0. 1. 1. 0. 1. 0. 0. 1. 0. 0.
 1. 1. 0. 0. 0. 0. 0. 1. 1. 0. 1. 1. 0. 0. 1. 0. 1. 0. 1. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1. 1. 0. 1. 1. 1. 0. 1. 0. 0. 1. 0. 1. 0. 0. 0. 0. 1. 0. 0.
 1. 0. 1. 0. 1. 0. 1. 0. 1. 1. 0. 1. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 1. 1.
 1. 1. 0. 0. 0. 0. 1. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 1.
 0. 0. 0. 0. 1. 0. 0. 0. 1. 1. 0. 1. 0. 0. 0. 0. 1. 1. 1. 1. 1. 0. 0. 0.
 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 1. 0. 0. 0. 0.
 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0.

In [89]:
d_reg = MyDecisionTreeRegressor(max_depth=5, min_samples_split=2)
d_reg.fit(diabetes_X,diabetes_Y)
d_prediction_reg = d_reg.predict(diabetes_X)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in double_scalars


[265.  72. 122. 131.  53.  72.  53.  66. 129. 122.  53. 129.  72. 131.
  72. 131. 265. 131.  53.  72.  53.  53.  72. 230. 129. 131.  72. 129.
 109. 292. 129.  72. 321.  72.  53. 170. 265. 131. 233. 131. 109.  53.
  53.  72. 122.  53.  53.  53.  66. 131.  53. 265.  72.  53. 129.  72.
 122.  72. 170. 170.  72. 131.  65.  53.  53. 170. 129.  53.  53.  53.
  72. 265. 131.  53. 109. 170. 191.  53. 252.  53. 131.  53.  72. 136.
  72. 129.  70. 131.  65.  72.  72. 122. 109.  72.  72. 170. 131. 264.
  53. 170. 122.  66. 302. 288. 129.  53.  53. 129. 265.  53.  66.  72.
 302. 292. 275. 131. 265. 292. 292. 170.  53. 265. 233. 122. 129. 131.
  72. 109.  72. 268. 321.  53.  53.  53. 129. 122.  72. 265. 321. 265.
 191. 268. 131.  72. 122. 288. 122. 233. 129. 131. 288.  72. 292.  53.
 122. 265.  53. 170.  53. 265.  53. 230. 129. 288. 131.  72.  70. 275.
 268. 170.  66.  53. 275.  53. 131.  53. 265. 122.  72. 109. 131.  72.
 131. 168. 122. 131. 122.  72. 109.  53. 131. 178.  66. 131. 170. 122.
  72. 

In [249]:
############ The results from sklearn official API ###########################
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(random_state=0)
clf.fit(titanic_X,titanic_Y)
prediction = clf.predict(titanic_X_test)
print(prediction)

[0 0 1 1 1 0 0 0 1 0 0 0 1 1 1 1 0 1 1 0 1 1 1 0 1 0 1 1 1 0 0 0 1 0 1 0 0
 0 0 1 0 0 0 1 1 0 1 0 1 1 0 0 1 0 0 0 0 0 0 1 0 0 0 1 1 1 1 0 0 1 0 0 1 0
 1 0 0 0 0 0 1 0 0 0 0 0 1 1 0 1 1 0 0 0 0 0 1 1 1 0 1 0 0 0 1 0 0 0 0 0 0
 0 1 1 1 0 0 1 0 1 1 0 1 0 0 1 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1 0 0 1 0 0
 0 0 1 0 0 1 0 0 1 1 0 1 0 1 1 0 0 1 0 0 1 0 0 0 0 0 0 1 1 1 1 1 0 1 1 0 1
 0 1 1 0 0 0 0 1 0 1 0 1 0 0 0 1 1 0 1 0 0 0 0 1 0 1 0 0 1 0 0 0 1 1 0 1 0
 1 1 1 0 0 0 0 0 0 1 0 0 0 0 0 1 1 1 1 1 1 0 0 0 1 0 1 0 1 0 0 0 0 0 0 0 0
 0 0 0 1 1 0 0 0 0 0 0 0 0 1 1 0 1 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 0 0 0 0 0 1 0 1 1 0 0 1 0 0 1 1 1 0 0 0 0 0 0 1 1 0 1 0 0 0 1 1 0
 1 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 1 1 0 0 0 1 0 1 0 0 1 0 1 1 1 1 1 0 1 1 0
 0 1 0 0 1 1 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 1 1 0 1 1 0 1 0 0 1 0 1 1 0 0 0
 0 0 1 0 1 1 0 1 0 0 1]


In [250]:
sum([d_prediction[i]==prediction[i] for i in range(len(prediction))])/len(prediction)

0.8253588516746412

In [90]:
from sklearn.tree import DecisionTreeRegressor

reg = DecisionTreeRegressor(random_state=0, max_depth=5)
reg.fit(diabetes_X,diabetes_Y)
prediction_reg = reg.predict(diabetes_X)
print(sum([abs(prediction_reg[i]-d_prediction_reg[i]) for i in range(len(prediction_reg))])/len(prediction_reg))

35.229329494035476


In [24]:
################################### More Test Cases ############################################
import math 

def divide_on_feature(X, feature_i, threshold):
    """ Divide dataset based on if sample value on feature index is larger than
        the given threshold """
    split_func = None
    if isinstance(threshold, int) or isinstance(threshold, float):
        split_func = lambda sample: sample[feature_i] >= threshold
    else:
        split_func = lambda sample: sample[feature_i] == threshold

    X_1 = np.array([sample for sample in X if split_func(sample)])
    X_2 = np.array([sample for sample in X if not split_func(sample)])

    return np.array([X_1, X_2])

def calculate_entropy(y):
    """ Calculate the entropy of label array y """
    log2 = lambda x: math.log(x) / math.log(2)
    unique_labels = np.unique(y)
    entropy = 0
    for label in unique_labels:
        count = len(y[y == label])
        p = count / len(y)
        entropy += -p * log2(p)
    return entropy

class DecisionNode():
    """Class that represents a decision node or leaf in the decision tree
    Parameters:
    -----------
    feature_i: int
        Feature index which we want to use as the threshold measure.
    threshold: float
        The value that we will compare feature values at feature_i against to
        determine the prediction.
    value: float
        The class prediction if classification tree, or float value if regression tree.
    true_branch: DecisionNode
        Next decision node for samples where features value met the threshold.
    false_branch: DecisionNode
        Next decision node for samples where features value did not meet the threshold.
    """
    def __init__(self, feature_i=None, threshold=None,
                 value=None, true_branch=None, false_branch=None):
        self.feature_i = feature_i          # Index for the feature that is tested
        self.threshold = threshold          # Threshold value for feature
        self.value = value                  # Value if the node is a leaf in the tree
        self.true_branch = true_branch      # 'Left' subtree
        self.false_branch = false_branch    # 'Right' subtree


# Super class of RegressionTree and ClassificationTree
class DecisionTree(object):
    """Super class of RegressionTree and ClassificationTree.
    Parameters:
    -----------
    min_samples_split: int
        The minimum number of samples needed to make a split when building a tree.
    min_impurity: float
        The minimum impurity required to split the tree further.
    max_depth: int
        The maximum depth of a tree.
    loss: function
        Loss function that is used for Gradient Boosting models to calculate impurity.
    """
    def __init__(self, min_samples_split=2, min_impurity=1e-7,
                 max_depth=float("inf"), loss=None):
        self.root = None  # Root node in dec. tree
        # Minimum n of samples to justify split
        self.min_samples_split = min_samples_split
        # The minimum impurity to justify split
        self.min_impurity = min_impurity
        # The maximum depth to grow the tree to
        self.max_depth = max_depth
        # Function to calculate impurity (classif.=>info gain, regr=>variance reduct.)
        self._impurity_calculation = None
        # Function to determine prediction of y at leaf
        self._leaf_value_calculation = None
        # If y is one-hot encoded (multi-dim) or not (one-dim)
        self.one_dim = None
        # If Gradient Boost
        self.loss = loss

    def fit(self, X, y, loss=None):
        """ Build decision tree """
        self.one_dim = len(np.shape(y)) == 1
        self.root = self._build_tree(X, y)
        self.loss=None

    def _build_tree(self, X, y, current_depth=0):
        """ Recursive method which builds out the decision tree and splits X and respective y
        on the feature of X which (based on impurity) best separates the data"""

        largest_impurity = 0
        best_criteria = None    # Feature index and threshold
        best_sets = None        # Subsets of the data

        # Check if expansion of y is needed
        if len(np.shape(y)) == 1:
            y = np.expand_dims(y, axis=1)

        # Add y as last column of X
        Xy = np.concatenate((X, y), axis=1)

        n_samples, n_features = np.shape(X)

        if n_samples >= self.min_samples_split and current_depth <= self.max_depth:
            # Calculate the impurity for each feature
            for feature_i in range(n_features):
                # All values of feature_i
                feature_values = np.expand_dims(X[:, feature_i], axis=1)
                unique_values = np.unique(feature_values)

                # Iterate through all unique values of feature column i and
                # calculate the impurity
                for threshold in unique_values:
                    # Divide X and y depending on if the feature value of X at index feature_i
                    # meets the threshold
                    Xy1, Xy2 = divide_on_feature(Xy, feature_i, threshold)

                    if len(Xy1) > 0 and len(Xy2) > 0:
                        # Select the y-values of the two sets
                        y1 = Xy1[:, n_features:]
                        y2 = Xy2[:, n_features:]

                        # Calculate impurity
                        impurity = self._impurity_calculation(y, y1, y2)

                        # If this threshold resulted in a higher information gain than previously
                        # recorded save the threshold value and the feature
                        # index
                        if impurity > largest_impurity:
                            largest_impurity = impurity
                            best_criteria = {"feature_i": feature_i, "threshold": threshold}
                            best_sets = {
                                "leftX": Xy1[:, :n_features],   # X of left subtree
                                "lefty": Xy1[:, n_features:],   # y of left subtree
                                "rightX": Xy2[:, :n_features],  # X of right subtree
                                "righty": Xy2[:, n_features:]   # y of right subtree
                                }

        if largest_impurity > self.min_impurity:
            # Build subtrees for the right and left branches
            true_branch = self._build_tree(best_sets["leftX"], best_sets["lefty"], current_depth + 1)
            false_branch = self._build_tree(best_sets["rightX"], best_sets["righty"], current_depth + 1)
            return DecisionNode(feature_i=best_criteria["feature_i"], threshold=best_criteria[
                                "threshold"], true_branch=true_branch, false_branch=false_branch)

        # We're at leaf => determine value
        leaf_value = self._leaf_value_calculation(y)

        return DecisionNode(value=leaf_value)


    def predict_value(self, x, tree=None):
        """ Do a recursive search down the tree and make a prediction of the data sample by the
            value of the leaf that we end up at """

        if tree is None:
            tree = self.root

        # If we have a value (i.e we're at a leaf) => return value as the prediction
        if tree.value is not None:
            return tree.value

        # Choose the feature that we will test
        feature_value = x[tree.feature_i]

        # Determine if we will follow left or right branch
        branch = tree.false_branch
        if isinstance(feature_value, int) or isinstance(feature_value, float):
            if feature_value >= tree.threshold:
                branch = tree.true_branch
        elif feature_value == tree.threshold:
            branch = tree.true_branch

        # Test subtree
        return self.predict_value(x, branch)

    def predict(self, X):
        """ Classify samples one by one and return the set of labels """
        y_pred = [self.predict_value(sample) for sample in X]
        return y_pred

    def print_tree(self, tree=None, indent=" "):
        """ Recursively print the decision tree """
        if not tree:
            tree = self.root

        # If we're at leaf => print the label
        if tree.value is not None:
            print (tree.value)
        # Go deeper down the tree
        else:
            # Print test
            print ("%s:%s? " % (tree.feature_i, tree.threshold))
            # Print the true scenario
            print ("%sT->" % (indent), end="")
            self.print_tree(tree.true_branch, indent + indent)
            # Print the false scenario
            print ("%sF->" % (indent), end="")
            self.print_tree(tree.false_branch, indent + indent)


class ClassificationTree(DecisionTree):
    def _calculate_information_gain(self, y, y1, y2):
        # Calculate information gain
        p = len(y1) / len(y)
        entropy = calculate_entropy(y)
        info_gain = entropy - p * \
            calculate_entropy(y1) - (1 - p) * \
            calculate_entropy(y2)

        return info_gain

    def _majority_vote(self, y):
        most_common = None
        max_count = 0
        for label in np.unique(y):
            # Count number of occurences of samples with label
            count = len(y[y == label])
            if count > max_count:
                most_common = label
                max_count = count
        return most_common

    def fit(self, X, y):
        self._impurity_calculation = self._calculate_information_gain
        self._leaf_value_calculation = self._majority_vote
        super(ClassificationTree, self).fit(X, y)


In [25]:
d_clf_1 = ClassificationTree(max_depth=5, min_samples_split=2)
d_clf_1.fit(titanic_X,titanic_Y)
d_prediction_1 = d_clf_1.predict(titanic_X_test)
print(d_prediction_1)

[0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0,

In [26]:
sum([d_prediction_1[i]==d_prediction[i] for i in range(len(d_prediction))])/len(d_prediction)

0.9784688995215312